# Expense Manager

## Importing Libraries 

In [1]:
import pygsheets
import pydrive
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)

In [2]:
gc = pygsheets.authorize(service_file='sheets-b4a2e84afc87.json')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1g7MaNS74UR86VXFnaaIEDrBskKfJ-2tBx_100zAK_oo/edit#gid=0')

## Data Processing

In [3]:
data = sh[0].get_as_df()

In [4]:
days = data['Date'].nunique()
gross_expense = data['Total Amount'].sum()
gross_per_person = data['Per Person'].sum()

TypeError: must be str, not int

In [ ]:
#Converting into Integer
numeric_columns = ['No. of Shares','Quantity/Number','Total Amount','Per Person']
for column_name in numeric_columns:
    data[column_name] = data[column_name].astype(float)

## Data Analysis

In [ ]:
#Day wise analysis
data_per_day = pd.DataFrame(columns=['Date','Total Amount','Per Person'])
for date in data['Date'].unique():
    daily_data = data[data['Date'] == date]
    total_amount = daily_data['Total Amount'].sum()
    total_per_person = daily_data['Per Person'].sum()
    data_per_day = data_per_day.append({'Date': date, 'Total Amount': total_amount,'Per Person': total_per_person},ignore_index=True)
# data_per_day

In [ ]:
#Item wise analysis
item_data = pd.DataFrame(columns=['Item','Frequency','Frequency/Days','Quantity','Quantity/Days','Total Amount','Per Person','Per Person/Days'])
for items in data.Item.unique():
    items_data = data[data['Item'] == items]
    frequency = items_data.shape[0]
    quantity = items_data['Quantity/Number'].sum()
    per_person_total = items_data['Per Person'].sum()
    total_amount = items_data['Total Amount'].sum()
    item_data = item_data.append({'Item':items,'Frequency': frequency,'Frequency/Days': frequency/days,'Quantity': quantity,'Quantity/Days': quantity/days,'Total Amount': total_amount,'Per Person': per_person_total,'Per Person/Days': per_person_total/days},ignore_index=True)
# item_data.sort_values(by=['Total Amount'])

In [ ]:
#Inserting the dataframes into GoogleSheet
sh[1].set_dataframe(data_per_day.round(2),'J2')
sh[1].set_dataframe(item_data.sort_values(by=['Total Amount']).round(2),'A2')
sh[1].cell('O3').set_value(days)
sh[1].cell('O4').set_value(round(gross_expense,2))
sh[1].cell('O5').set_value(round(gross_per_person,2))
sh[1].cell('O6').set_value(round(gross_per_person/days,2))

## Plotting

In [ ]:
data_processed = sh[1].get_as_df()

In [ ]:
def gen_perday_values(df_column,if_int):
    holder_list = list()
    for value in df_column:
        if value != None:
            if if_int:
                holder_list.append(int(value))
            else:
                holder_list.append(value)
    holder_list.reverse()
    return holder_list

In [ ]:
def gen_moving_avg(list_values):
    holder_list = list()
    n = 1
    total = 0
    for value in list_values:
        total += value
        avg = total/n
        holder_list.append(avg)
        n += 1
    return holder_list

In [ ]:
dates = gen_perday_values(data_processed.iloc[1:,9],0)
TPD = gen_perday_values(data_processed.iloc[1:,10],1)
TPP = gen_perday_values(data_processed.iloc[1:,11],1)
TPD_avg = gen_moving_avg(TPD)
TPP_avg = gen_moving_avg(TPP)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
values = TPD
labels = dates

plt.plot(labels, values,color='orange')
plt.plot(TPD_avg,color='r')

for i, v in enumerate(values):
    ax.text(i, v+15, "%d" %v, ha="center",fontsize=13)
# plt.ylim(-10, 595)

for i, v in enumerate(TPD_avg):
    ax.text(i, v+15, "%d" %v, ha="center",fontsize=13)
# plt.ylim(-10, 595)

plt.title('Total Per Day Expense',pad=10,fontsize=20)
plt.xlabel('Dates',fontsize=15)
plt.ylabel('Amount',fontsize=15)
plt.xticks(rotation=45,fontsize=13)
plt.yticks(fontsize=13)
plt.grid(True)
daily_line = matplotlib.patches.Patch(color='orange', label='Daily Values')
avg_line = matplotlib.patches.Patch(color='red', label='Moving Average')
plt.legend(handles=[daily_line,avg_line],fontsize=13,loc=2)
plt.savefig('./TPD.png')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
values = TPP
labels = dates

plt.plot(labels, values,color='orange')
plt.plot(TPP_avg,color='red')

for i, v in enumerate(values):
    ax.text(i, v+15, "%d" %v, ha="center",fontsize=13)
plt.ylim(-10, 595)

for i, v in enumerate(TPP_avg):
    ax.text(i, v+15, "%d" %v, ha="center",fontsize=13)
plt.ylim(-10, 595)

plt.title('Total Per Person Expense',pad=10,fontsize=20)
daily_line = matplotlib.patches.Patch(color='orange', label='Daily Values')
avg_line = matplotlib.patches.Patch(color='red', label='Moving Average')
plt.legend(handles=[daily_line,avg_line],fontsize=13,loc=2)
plt.grid(True)
plt.xlabel('Dates',fontsize=15)
plt.ylabel('Amount',fontsize=15)
plt.xticks(rotation=45,fontsize=13)
plt.yticks(fontsize=13)
plt.savefig('./TPP.png')
plt.show()

In [ ]:
Others_df = pd.DataFrame(item_data.sort_values(by=['Total Amount']).iloc[0:-10,:].sum()).T
Others_df['Item'][0] = 'Others'
top_10_df = item_data.sort_values(by=['Total Amount']).iloc[-10:]
top_10_df = top_10_df.append(Others_df)

In [ ]:
top_10_df.sort_values(by=['Total Amount']).iloc[-10:].set_index(top_10_df.sort_values(by=['Total Amount']).iloc[-10:]['Item']).plot.pie(y='Total Amount',legend=None,autopct='%1.0f%%',pctdistance=0.9,fontsize=15)
plt.title('Top 10 Items',fontsize=20)
plt.savefig('./top_10_pie.png')
plt.show()